In [1]:
import pandas as pd
import numpy as np
import cx_Oracle
from py2neo import Graph, Node, Relationship, NodeMatcher
import warnings

# Desabilitar o aviso temporariamente
warnings.filterwarnings("ignore", category=FutureWarning)

# Conexões

In [2]:
instant_client = "./instantclient_21_9"
cx_Oracle.init_oracle_client(lib_dir=instant_client)
# Conectar ao banco de dados Oracle
conn = cx_Oracle.connect(user="store", password="pass", dsn="localhost:1521/xe")

# estabelecer conexão neo4j
graph = Graph("bolt://localhost:7687 ", auth=("neo4j", "password"))
matcher = NodeMatcher(graph)

# SQL

In [3]:
tabelasSQL = ["STORE_USERS", "PRODUCT_CATEGORIES", "PRODUCT", "DISCOUNT", "CART_ITEM", "SHOPPING_SESSION", "ORDER_DETAILS", "ORDER_ITEMS", "PAYMENT_DETAILS", "EMPLOYEES", "DEPARTMENTS", "ADRESSES", "EMPLOYEES_ARCHIVE", "STOCK"]

# Product
product_categories_fields = ["category_id", "category_name"]
product_fields = ["product_id", "product_name", "category_id", "sku", "price", "discount_id", "created_at", "last_modified"]
discount_fields = ["discount_id", "discount_name", "discount_desc", "discount_percent", "is_active_status", "created_at", "modified_at"]
stock_fields = ["product_id", "quantity", "max_stock_quantity", "unit"]

# Employees
employees_fields = ["employee_id", "first_name", "middle_name", "last_name", "date_of_birth", "department_id", "hire_date", "salary", "phone_number", "email", "ssn_number", "manager_id"]
departments_fields = ["department_id", "department_name", "manager_department_id", "department_desc"]

# Empolyees Archive
employees_archive_fields = ["event_date", "event_type", "user_name", "old_employee_id", "old_first_name", "old_middle_name", "old_last_name", "old_date_of_birth", "old_department_id", "old_hire_date", "old_salary", "old_phone_number", "old_email", "old_ssn_number", "old_manager_id", "new_employee_id", "new_first_name", "new_middle_name", "new_last_name", "new_date_of_birth", "new_department_id", "new_hire_date", "new_salary", "new_phone_number", "new_email", "new_ssn_number", "new_manager_id"]

# ORDER_DETAILS
order_details_fields = ["order_details_id", "user_id", "total", "payment_id", "shipping_method", "delivery_adress_id", "created_at", "modified_at"]
#                          payment_details_id
payment_details_fields = ["payment_id", "order_details_id", "amount", "provider", "payment_status", "payment_created_at", "payment_modified_at"]

# ADDRESES
addresses_fields = ["adress_id", "line_1", "line_2", "city", "zip_code", "province", "country"]

# STORE_USERS
store_users_fields = ["user_id", "first_name", "middle_name", "last_name", "phone_number", "email", "username", "user_password", "registered_at"]

# CART_ITEM
cart_item_fields = ["cart_item_id", "session_id", "product_id", "quantity", "created_at", "modified_at"]

# SHOPPING_SESSION
shopping_session_fields = ["session_id", "user_id", "created_at", "modified_at"]

# ORDER_ITEMS
order_items_fields = ["order_items_id", "order_details_id", "product_id", "created_at", "modified_at"]

In [4]:
# Criar um cursor
cursor = conn.cursor()

### PRODUCT CATEGORIES

In [5]:
cursor.execute('SELECT * FROM PRODUCT_CATEGORIES')
product_categories = cursor.fetchall()

product_categories_df = pd.DataFrame(product_categories)
product_categories_df.columns = product_categories_fields

### PRODUCT

#### PRODUCT

In [6]:
cursor.execute('SELECT * FROM PRODUCT')
product = cursor.fetchall()

product_df = pd.DataFrame(product)
product_df.columns = product_fields

#### DISCOUNT

In [7]:
cursor.execute('SELECT * FROM DISCOUNT')
discount = cursor.fetchall()

discount_df = pd.DataFrame(discount)
discount_df.columns = discount_fields

nanLinha = pd.Series(np.nan, index=discount_fields)
discount_df = discount_df.append(nanLinha, ignore_index=True)

In [8]:
productNeo_df = pd.merge(product_df, discount_df, on='discount_id')
productNeo_df = productNeo_df.loc[:, ~productNeo_df.columns.duplicated()]
productNeo_df = productNeo_df.replace({None: np.nan})

#### STOCK

In [9]:
cursor.execute('SELECT * FROM STOCK')
stock = cursor.fetchall()

stock_df = pd.DataFrame(stock)
stock_df.columns = stock_fields

In [10]:
productNeo_df = pd.merge(productNeo_df, stock_df, on='product_id')

### EMPLOYEES

In [11]:
cursor.execute('SELECT * FROM EMPLOYEES')
employees = cursor.fetchall()

employees_df = pd.DataFrame(employees)
employees_df.columns = employees_fields

### DEPARTMENT

In [12]:
cursor.execute('SELECT * FROM DEPARTMENTS')
departments = cursor.fetchall()

departments_df = pd.DataFrame(departments)
departments_df.columns = departments_fields

### EMPLOYEES_ARCHIVE

In [13]:
cursor.execute('SELECT * FROM EMPLOYEES_ARCHIVE')
employees_archive = cursor.fetchall()

employees_archive_df = pd.DataFrame(employees_archive)
employees_archive_df.columns = employees_archive_fields

### STORE_USERS

In [14]:
cursor.execute('SELECT * FROM STORE_USERS')
store_users = cursor.fetchall()

store_users_df = pd.DataFrame(store_users)
store_users_df.columns = store_users_fields

### SHOPPING_SESSION

In [15]:
cursor.execute('SELECT * FROM SHOPPING_SESSION')
shopping_session = cursor.fetchall()

shopping_session_df = pd.DataFrame(shopping_session)
shopping_session_df.columns = shopping_session_fields

### CART_ITEM

In [16]:
cursor.execute('SELECT * FROM CART_ITEM')
cart_item = cursor.fetchall()

cart_item_df = pd.DataFrame(cart_item)
cart_item_df.columns = cart_item_fields

### ORDER_DETAILS

In [17]:
cursor.execute('SELECT * FROM ORDER_DETAILS')
order_details = cursor.fetchall()

order_details_df = pd.DataFrame(order_details)
order_details_df.columns = order_details_fields

#### PAYMENT_DETAILS

In [18]:
cursor.execute('SELECT * FROM PAYMENT_DETAILS')
payment_details = cursor.fetchall()

payment_details_df = pd.DataFrame(payment_details)
payment_details_df.columns = payment_details_fields

In [19]:
order_details_neo_df = pd.merge(order_details_df, payment_details_df, on='payment_id')
order_details_neo_df = order_details_neo_df.loc[:, ~order_details_neo_df.columns.duplicated()]
order_details_neo_df.drop("order_details_id_y", axis=1, inplace=True)
order_details_neo_df.rename(columns={"order_details_id_x": "order_details_id"}, inplace=True) 

### ADDRESSES

In [20]:
cursor.execute('SELECT * FROM ADDRESSES')
addresses = cursor.fetchall()

addresses_df = pd.DataFrame(addresses)
addresses_df.columns = addresses_fields

### ORDER_ITEMS

In [21]:
cursor.execute('SELECT * FROM ORDER_ITEMS')
order_items = cursor.fetchall()

order_items_df = pd.DataFrame(order_items)
order_items_df.columns = order_items_fields

# Inserir NEO4J

### PRODUCT_CATEGORIES

In [22]:
product_categories_dict = product_categories_df.to_dict(orient='records')

product_categories_dict_rel = {}
for pc in product_categories_dict:
    product_categories_node = Node("PRODUCT_CATEGORIES",  category_id=pc['category_id'], category_name=pc['category_name'])
    graph.create(product_categories_node)
    product_categories_dict_rel[pc['category_id']] = product_categories_node

### PRODUCT

In [23]:
productNeo_df.replace({pd.NaT: None}, inplace=True)

product_dict = productNeo_df.to_dict(orient='records')
product_dict_rel = {}
for p in product_dict:
    product_node = Node("PRODUCT", product_id=p["product_id"], product_name=p["product_name"], category_id=p["category_id"], sku=p["sku"], price=p["price"], discount_id=p["discount_id"], created_at_x=p["created_at_x"], last_modified=p["last_modified"], discount_name=p["discount_name"], discount_desc=p["discount_desc"], discount_percent=p["discount_percent"], is_active_status=p["is_active_status"], created_at_y=p["created_at_y"], modified_at=p["modified_at"], quantity=p["quantity"], max_stock_quantity=p["max_stock_quantity"], unit=p["unit"])
    graph.create(product_node)
    
    pc_p_rel = Relationship(product_node, "category", product_categories_dict_rel[p["category_id"]])
    graph.create(pc_p_rel)
    product_dict_rel[p["product_id"]] = product_node

### DEPARTMENT

In [24]:
departments_dict = departments_df.to_dict(orient='records')
departments_dict_rel = {}

for dp in departments_dict:
    department_node = Node("DEPARTMENT", department_id=dp["department_id"], department_name=dp["department_name"])
    graph.create(department_node)
    departments_dict_rel[dp["department_id"]] = department_node

### EMPLOYEES

In [25]:
employees_dict = employees_df.to_dict(orient='records')

for e in employees_dict:
    employees_node = Node("EMPLOYEES", employee_id=e["employee_id"], first_name=e["first_name"], middle_name=e["middle_name"], last_name=e["last_name"], date_of_birth=e["date_of_birth"], department_id=e["department_id"], hire_date=e["hire_date"], salary=e["salary"], phone_number=e["phone_number"], email=e["email"], ssn_number=e["ssn_number"], manager_id=e["manager_id"])
    graph.create(employees_node)

    e_dp_rel = Relationship(employees_node, "work", departments_dict_rel[e["department_id"]])
    graph.create(e_dp_rel)

### EMPLOYEES_ARCHIVE

In [26]:
employees_archive_dict = employees_archive_df.to_dict(orient='records')

for ea in employees_archive_dict:
    employee_archive_node = Node("EMPLOYEE_ARCHIVE", event_date=ea["event_date"], event_type=ea["event_type"], user_name=ea["user_name"], old_employee_id=ea["old_employee_id"], old_first_name=ea["old_first_name"], old_middle_name=ea["old_middle_name"], old_last_name=ea["old_last_name"], old_date_of_birth=ea["old_date_of_birth"], old_department_id=ea["old_department_id"], old_hire_date=ea["old_hire_date"], old_salary=ea["old_salary"], old_phone_number=ea["old_phone_number"], old_email=ea["old_email"], old_ssn_number=ea["old_ssn_number"], old_manager_id=ea["old_manager_id"], new_employee_id=ea["new_employee_id"], new_first_name=ea["new_first_name"], new_middle_name=ea["new_middle_name"], new_last_name=ea["new_last_name"], new_date_of_birth=ea["new_date_of_birth"], new_department_id=ea["new_department_id"], new_hire_date=ea["new_hire_date"], new_salary=ea["new_salary"], new_phone_number=ea["new_phone_number"], new_email=ea["new_email"], new_ssn_number=ea["new_ssn_number"], new_manager_id=ea["new_manager_id"])
    
    graph.create(employee_archive_node)

### STORE USERS

In [27]:
store_users_dict = store_users_df.to_dict(orient='records')
store_users_dict_rel = {}

for su in store_users_dict:
    store_users_node = Node("STORE_USERS", user_id=su["user_id"], first_name=su["first_name"], middle_name=su["middle_name"], last_name=su["last_name"], phone_number=su["phone_number"], email=su["email"], username=su["username"], user_password=su["user_password"], registered_at=su["registered_at"])

    graph.create(store_users_node)
    store_users_dict_rel[su["user_id"]] = store_users_node


### ADDRESSES

In [28]:
addresses_dict = addresses_df.to_dict(orient='records')
addresses_dict_rel = {}

for ad in addresses_dict:
    addresses_node = Node("ADDRESSES", adress_id=ad["adress_id"], line_1=ad["line_1"], line_2=ad["line_2"], city=ad["city"], zip_code=ad["zip_code"], province=ad["province"], country=ad["country"])
    graph.create(addresses_node)

    addresses_dict_rel[ad["adress_id"]] = addresses_node

### ORDER DETAILS

In [29]:
order_details_neo_df.replace({pd.NaT: None}, inplace=True)

order_details_dict = order_details_neo_df.to_dict(orient='records')
order_details_dict_rel = {}

for od in order_details_dict:
    order_details_node = Node("ORDER_DETAILS", order_details_id=od["order_details_id"], user_id=od["user_id"], total=od["total"], payment_id=od["payment_id"], shipping_method=od["shipping_method"], delivery_adress_id=od["delivery_adress_id"], created_at=od["created_at"], modified_at=od["modified_at"], amount=od["amount"], provider=od["provider"], payment_status=od["payment_status"], payment_created_at=od["payment_created_at"], payment_modified_at=od["payment_modified_at"])
    graph.create(order_details_node)

    # RELATION ADDRESSES ORDER_DETAILS
    addr_order_rel = Relationship(order_details_node, "delivery", addresses_dict_rel[od["delivery_adress_id"]])
    graph.create(addr_order_rel)

    user_order_rel = Relationship(order_details_node, "buy", store_users_dict_rel[od["user_id"]])
    graph.create(user_order_rel)

    order_details_dict_rel[od["order_details_id"]] = order_details_node

### SHOPPING SESSION

In [30]:
shopping_session_dict = shopping_session_df.to_dict(orient='records')
shopping_session_dict_rel = {}

for ss in shopping_session_dict:
    shopping_session_node = Node("SHOPPING_SESSION", session_id=ss["session_id"], user_id=ss["user_id"], created_at=ss["created_at"], modified_at=ss["modified_at"])
    graph.create(shopping_session_node)

    user_session_rel = Relationship(order_details_node, "session", store_users_dict_rel[ss["user_id"]])
    graph.create(user_session_rel)

    order_details_dict_rel[ss["session_id"]] = shopping_session_node

### CART_ITEM

In [46]:
cart_item_df.replace({pd.NaT: None}, inplace=True)
cart_item_dict = cart_item_df.to_dict(orient='records')

for ci in cart_item_dict:
    cart_item_rel = Relationship(order_details_dict_rel[ci["session_id"]], "cart_item", product_dict_rel[ci["product_id"]],cart_item_id=ci["cart_item_id"], session_id=ci["session_id"], product_id=ci["product_id"], quantity=ci["quantity"], created_at=ci["created_at"], modified_at=ci["modified_at"])

    graph.create(cart_item_rel)

### ORDER ITEMS

In [34]:
order_items_df.replace({pd.NaT: None}, inplace=True)

order_items_dict = order_items_df.to_dict(orient='records')
for oi in order_items_dict:
    order_items_rel = Relationship(order_details_dict_rel[oi["order_details_id"]], "order_items", product_dict_rel[oi["product_id"]], order_items_id=oi["order_items_id"], order_details_id=oi["order_details_id"], product_id=oi["product_id"], created_at=oi["created_at"], modified_at=oi["modified_at"])

    graph.create(order_items_rel)